<a href="https://colab.research.google.com/github/Itsuki-Hamano123/Quantum_prac/blob/master/ising_3to2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/量子コンピュータ練習/ナレッジ用に解く

/content/drive/My Drive/量子コンピュータ練習/ナレッジ用に解く


#組み合わせ最適化問題を定式化する際に起きる多体問題を、補助変数を使って２体問題に落とし込む

東工大の西森先生の「[量子アニーリングの基礎](https://www.amazon.co.jp/%E9%87%8F%E5%AD%90%E3%82%A2%E3%83%8B%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E3%81%AE%E5%9F%BA%E7%A4%8E-%E5%9F%BA%E6%9C%AC%E6%B3%95%E5%89%87%E3%81%8B%E3%82%89%E8%AA%AD%E3%81%BF%E8%A7%A3%E3%81%8F%E7%89%A9%E7%90%86%E5%AD%A6%E6%9C%80%E5%89%8D%E7%B7%9A-18-%E8%A5%BF%E6%A3%AE-%E7%A7%80%E7%A8%94/dp/4320035380/ref=sr_1_1?adgrpid=52917110065&gclid=EAIaIQobChMI4L2Fv4un4gIVVqWWCh0lbQ_9EAAYASAAEgIh5_D_BwE&hvadid=338550746380&hvdev=c&hvlocphy=1009332&hvnetw=g&hvpos=1t1&hvqmt=e&hvrand=12758953271038180148&hvtargid=kwd-463635452634&hydadcr=4073_10899429&jp-ad-ap=0&keywords=%E9%87%8F%E5%AD%90%E3%82%A2%E3%83%8B%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E3%81%AE%E5%9F%BA%E7%A4%8E&qid=1558251615&s=gateway&sr=8-1)」という本を購入しました。
その本の中で、問題を定式化した際に起きる多体問題を2体問題に落とし込む方法が記載してあったので、実際に試してみようと思います。
***
**今回の解きたい最適化問題**<br>
以前の記事で多体問題となった、$ab=3$を満たすときの整数$a,b$の組み合わせを探します。<br>
**解く手順**<br>
1. 問題をコスト関数に定式化
2. 補助変数を使って多体問題を2体問題に落とし込む
3. コスト関数（問題と補助変数の制約条件）をQUBOに変換
4. SAで最適解を求める

***

# 問題をコスト関数に定式化
$ab=3$を満たす整数$a,b$の組み合わせを探すため、最小二乗法を使って以下のコスト関数にします。
$0=\{(a*b)-3\}^2$

各量子ビットはバイナリ値を取るため、整数値を表現できるよう$a=q_0+2q_1$、$b=q_2+2q_3$とおき、先ほどのコスト関数に代入します。
$0=[\{(q_0+2q_1)*(q_2+2q_3)\}-3]^2$

式を展開します。


In [0]:
from sympy import *
q0,q1,q2,q3=symbols("q0 q1 q2 q3")
f=((q0+2*q1)*(q2+2*q3)-3)**2
print(expand(f))#式の展開

q0**2*q2**2 + 4*q0**2*q2*q3 + 4*q0**2*q3**2 + 4*q0*q1*q2**2 + 16*q0*q1*q2*q3 + 16*q0*q1*q3**2 - 6*q0*q2 - 12*q0*q3 + 4*q1**2*q2**2 + 16*q1**2*q2*q3 + 16*q1**2*q3**2 - 12*q1*q2 - 24*q1*q3 + 9


$q_0^2q_2^2+4q_0^2q_2q_3+4q_0^2q_3^2+4q_0q_1q_2^2+16q_0q_1q_2q_3+16q_0q_1q_3^2-6q_0q_2-12q_0q_3+4q_1^2q_2^2+16q_1^2q_2q_3+16q_1^2q_3^2-12q_1q_2-24q_1q_3+9$<br>
各量子ビットがバイナリ値を取り、二乗しても値が変わらない（$q_i=q_i^2$）ため、指数部が取れます。

$q_0q_2+4q_0q_2q_3+4q_0q_3+4q_0q_1q_2+16q_0q_1q_2q_3+16q_0q_1q_3-6q_0q_2-12q_0q_3+4q_1q_2+16q_1q_2q_3+16q_1q_3-12q_1q_2-24q_1q_3+9$

式を整理
$-5q_0q_2+4q_0q_2q_3-8q_0q_3+4q_0q_1q_2+16q_0q_1q_2q_3+16q_0q_1q_3-8q_1q_2+16q_1q_2q_3-8q_1q_3+9$<br>

#補助変数を使って多体問題を２体問題に落とし込む

例えば、３体相互作用$q_0q_1q_2$を２体問題に置き換えたいとした時、積$q_0q_1$を新たな二値変数$q_4$（=0,1）で置き換えて、$q_4q_2$と書けば見かけ上、２体問題になります。ただし、$q_0q_1$と$q_4$が等しくなるように、$q_0q_1=q_4$が成り立たないとエネルギーが高くなって基底状態から外れるような制約条件を考えます。<br>

|$q_0$|$q_1$|$q_4$|$H_p$|
|:-:|:-:|:-:|:-:|
|0|0|0|0|
|1|0|0|0|
|0|1|0|0|
|1|1|0|1|
|0|0|1|3|
|0|1|1|1|
|1|0|1|1|
|1|1|1|0|


制約条件のコスト関数
$H_p=3q_4+q_0q_1-2q_0q_4-2q_1q_4$

（同様に積$q_2q_3$も新たな変数$q_5$に置き換えた時の制約条件のコスト関数）
$H_p=3q_5+q_2q_3-2q_2q_5-2q_3q_5$

問題のコスト関数
$-5q_0q_2+4q_0q_2q_3-8q_0q_3+4q_0q_1q_2+16q_0q_1q_2q_3+16q_0q_1q_3-8q_1q_2+16q_1q_2q_3-8q_1q_3+9$

に、$q_0q_1=q_4,q_2q_3=q_5$を代入します。


$-5q_0q_2+4q_0q_5-8q_0q_3+4q_2q_4+16q_4q_5+16q_3q_4-8q_1q_2+16q_1q_5-8q_1q_3+9$

# それぞれのコスト関数をQUBOに変換
問題のコスト関数をQUBOに変換

||$q_0$|$q_1$|$q_2$|$q_3$|$q_4$|$q_5$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$q_0$|0|0|-5|-8|0|4|
|$q_1$|0|0|-8|-8|0|16|
|$q_2$|0|0|0|0|4|0|
|$q_3$|0|0|0|0|16|0|
|$q_4$|0|0|0|0|0|16|
|$q_5$|0|0|0|0|0|0|

制約条件のコスト関数をQUBOに変換

||$q_0$|$q_1$|$q_2$|$q_3$|$q_4$|$q_5$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$q_0$|0|1|0|0|-2|0|
|$q_1$|0|0|0|0|-2|0|
|$q_2$|0|0|0|1|0|-2|
|$q_3$|0|0|0|0|0|-2|
|$q_4$|0|0|0|0|3|0|
|$q_5$|0|0|0|0|0|3|

# ２つのQUBOを組み合わせSAで組み合わせを求める

In [0]:
!pip install wildqat

In [0]:
import wildqat as wq
import numpy as np
a = wq.opt()

qubo1 = np.array([[0,0,-5,-8,0,4],[0,0,-8,-8,0,16],[0,0,0,0,4,0],[0,0,0,0,16,0],[0,0,0,0,0,16],[0,0,0,0,0,0]])
qubo2=np.array([[0,1,0,0,-2,0],[0,0,0,0,-2,0],[0,0,0,1,0,-2],[0,0,0,0,0,-2],[0,0,0,0,3,0],[0,0,0,0,0,3]])

qubo=(0.01*qubo1)+0.7*qubo2
#qubo=qubo1+qubo2
print(qubo)

[[ 0.    0.7  -0.05 -0.08 -1.4   0.04]
 [ 0.    0.   -0.08 -0.08 -1.4   0.16]
 [ 0.    0.    0.    0.7   0.04 -1.4 ]
 [ 0.    0.    0.    0.    0.16 -1.4 ]
 [ 0.    0.    0.    0.    2.1   0.16]
 [ 0.    0.    0.    0.    0.    2.1 ]]


In [0]:
a.qubo=qubo
result=a.run(shots=100,sampler="fast",verbose=True)
wq.counter(result)

0 : [1, 0, 1, 1, 0, 1]
1 : [1, 1, 1, 0, 1, 0]
2 : [1, 0, 1, 1, 0, 1]
3 : [0, 1, 1, 0, 0, 0]
4 : [1, 1, 1, 0, 1, 0]
5 : [1, 0, 1, 1, 0, 1]
6 : [1, 0, 0, 1, 0, 0]
7 : [1, 0, 0, 1, 0, 0]
8 : [1, 1, 1, 0, 1, 0]
9 : [1, 0, 0, 1, 0, 0]
10 : [0, 1, 0, 1, 0, 0]
11 : [1, 0, 1, 0, 0, 0]
12 : [0, 1, 0, 1, 0, 0]
13 : [1, 0, 1, 1, 0, 1]
14 : [1, 0, 1, 1, 0, 1]
15 : [1, 0, 1, 1, 0, 1]
16 : [1, 0, 1, 1, 0, 1]
17 : [1, 0, 0, 1, 0, 0]
18 : [1, 0, 1, 1, 0, 1]
19 : [1, 0, 0, 1, 0, 0]
20 : [0, 1, 0, 1, 0, 0]
21 : [1, 1, 1, 0, 1, 0]
22 : [1, 0, 1, 1, 0, 1]
23 : [0, 1, 1, 0, 0, 0]
24 : [1, 0, 1, 1, 0, 1]
25 : [1, 0, 1, 1, 0, 1]
26 : [0, 0, 0, 1, 0, 0]
27 : [1, 0, 1, 0, 0, 0]
28 : [1, 0, 0, 1, 0, 0]
29 : [1, 0, 0, 1, 0, 0]
30 : [0, 1, 1, 0, 0, 0]
31 : [1, 1, 1, 0, 1, 0]
32 : [1, 0, 0, 1, 0, 0]
33 : [1, 0, 1, 1, 0, 1]
34 : [1, 1, 1, 1, 1, 1]
35 : [1, 0, 1, 1, 0, 1]
36 : [0, 1, 1, 0, 0, 0]
37 : [1, 0, 1, 0, 0, 0]
38 : [1, 0, 1, 1, 0, 1]
39 : [1, 1, 1, 0, 1, 0]
40 : [0, 1, 1, 0, 0, 0]
41 : [1, 0, 1, 0, 0, 0]
42

Counter({'000100': 1,
         '010100': 16,
         '011000': 12,
         '100100': 17,
         '101000': 8,
         '101101': 25,
         '110110': 1,
         '111010': 18,
         '111111': 2})

最も多く観測されたのは、25回の'101101'と、次に多く観測されたのは'、18回の111010'でした。<br>
今回の解きたい問題であった$ab=3$と、補助変数の制約条件$q_0q_1=q_4,q_2q_3=q_5$を満たしていることが分かります。<br>
**この結果から、補助変数を使うと多体問題を２体問題に落とし込めることが分かります。<br>
しかし、余計な結果もそれなりに出現しており、補助変数を設けることで少し精度が落ちるみたいです。そのためコスト関数を組み合わせる際のハイパーパラメータの調整が、よりシビアになり、ハイパーパラメータの調整に最適化手法を用いてあげる必要がありそうです。**